## Classifier and preprocessing

In this notebook, the noironicos dataset will be treated, since ironicos's tweets are all ironic and we want a mixture of ironic and non ironic.

In [3]:
# General import and load data
import numpy as np
import nltk
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
import re

# Needed for running
nltk.download('punkt')
nltk.download('stopwords')

# Import database
df=pd.read_csv('final_dataset.csv')
df.groupby('ironic').size()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ironic
0    5446
1    5638
dtype: int64

In [6]:
# Define X and Y
X = df['tweet'].values
y = df['ironic'].values.astype(int)


array([ 'Algunas personas sufren en las discos mientras tonean y se divierten a mil. #sarcasmo',
       '@jacevedoaraya es para sostener el marcador..... #sarcasmo',
       'Alguna de estas imágenes te sacara una sonrisa. \n#Humor #Sarcasmo #ViveFeliz #domingo #Relax #Diversión \nhttp://t.co/LqmV8iRO6V',
       ..., 'David Trueba tiene twitter?',
       '?Que opinais?: Manana, mi #dospalabras en @elconfidencial "La unica alternativa de Rajoy: sacar a Espana del Euro"',
       '#SahelNOW ahora en el telediario de Pepa Bueno'], dtype=object)

## Lexical features
The lexical features analysis will be performed by using the twitter tokenizer provided by nltk library.


In [50]:
# Sample statistics using NLTK
# A transformer will be implemented

from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import sent_tokenize, word_tokenize

class LexicalStats (BaseEstimator, TransformerMixin):
    """Extract lexical features from each document"""
    
    def number_sentences(self, doc):
        sentences = sent_tokenize(doc, language='spanish')
        return len(sentences)

    def fit(self, x, y=None):
        return self

    def transform(self, docs):
        return [{'length': len(doc),
                 'num_sentences': self.number_sentences(doc)}
                for doc in docs]

In [62]:
# A tokenizer will be defined
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string

def custom_tokenizer(words):
    tokens = word_tokenize(words.lower())
    stemmer = SnowballStemmer('spanish')
    lemmas = [stemmer.stem(t) for t in tokens]
    stoplist = stopwords.words('spanish')
    lemmas_clean = [w for w in lemmas if w not in stoplist]
    punctuation = set(string.punctuation)
    lemmas_punct = [w for w in lemmas_clean if  w not in punctuation]
    print(tokens)
    return lemmas_punct



['@', '_eurovision2014', 'en', '2013', 'falta', 'esdm', 'jajajajajajajajajajja', 'top', '1', 'hombre', '#', 'ironia']


['_eurovision2014',
 '2013',
 'falt',
 'esdm',
 'jajajajajajajajajajj',
 'top',
 '1',
 'hombr',
 'ironi']

## Syntactic features

ALOMEJOR HAY QUE QUITARLO

In [71]:
# We will use NLTK's tag set
from nltk import pos_tag, word_tokenize
import collections

# We can extract particular chunks (trozos, pedazos) from the sentence
# if we use a RegExpParser. See Syntactic Processing
def PosStats(BaseEstimator, TransformerMixin):
    
    def stats(self, doc):
        tokens = custom_tokenizer(doc)
        
        tagged = pos_tag(tokens, tagset = 'universal' )
        counts = collections.Counter(tag for word, tag in tagged)
        total = sum(counts.values())
        #copy tags so that we return always the same number of features
        pos_features = {'NOUN': 0, 'ADJ': 0, 'VERB': 0, 'ADV': 0, 'CONJ': 0, 
                        'ADP': 0, 'PRON':0, 'NUM': 0}
        
        pos_dic = dict((tag, float(count)/total) for tag,count in counts.items())
        for k in pos_dic:
            if k in pos_features:
                pos_features[k] = pos_dic[k]
        return pos_features
    
    def transform(self, docs, y=None):
        return [self.stats(doc) for doc in docs]
    
    def fit(self, docs, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self
        

## Feature extraction Pipeline
The feature extraction will be carried out by using pipelines. The defined pipelines are selected in order to extract the desired features

In [72]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer


ngrams_featurizer = Pipeline([
  ('count_vectorizer',  CountVectorizer(ngram_range = (1, 4), encoding = 'ISO-8859-1', 
                                        tokenizer=custom_tokenizer)),
  ('tfidf_transformer', TfidfTransformer())
])

## Feature Union Pipeline
Now we define which features we want to extract, how to combine them and later apple machine learning in the resulting feature set.

In [75]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def Pipeline(clf):
    pipeline = Pipeline([
           ('features', FeatureUnion([
                        ('lexical_stats', Pipeline([
                                    ('stats', LexicalStats()),
                                    ('vectors', DictVectorizer())
                                ])),
                        ('words', TfidfVectorizer(tokenizer=custom_tokenizer)),
                        ('ngrams', ngrams_featurizer),
                        ('pos_stats', Pipeline([
                                    ('pos_stats', PosStats()),
                                    ('vectors', DictVectorizer())
                                ])),
                        ('lda', Pipeline([ 
                                    ('count', CountVectorizer(tokenizer=custom_tokenizer)),
                                    ('lda',  LatentDirichletAllocation(n_topics=4, max_iter=5,
                                                           learning_method='online', 
                                                           learning_offset=50.,
                                                           random_state=0))
                                ])),
                    ])),

            ('clf', clf)  # classifier
        ])

# Using KFold validation

cv = KFold(X.shape[0], 2, shuffle=True, random_state=33)
scores = cross_val_score(pipeline, X, y, cv=cv)
print("Scores in every iteration", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Yo quiero hacer el confussion matrix y f score. Para ello, necesito el training y testing dataset. Eso lo puedo hacer usando el vector X, pero el vector X tiene strings (arrays de twits) y el modelo (osea mi pipeline) transforma ese array de strings en números. Esos números, son procesados y metidos en el pipeline para darme el modelo. Entonces, como quiero hacer el f1 score, necesito un training y testing dataset. Pero, paa hacer el método pipeline.predict(X_test) necesito que X_test sea números (sino salta error que ya me ha pasado). Entonces, como hago ese test split en el pipeline?

## Train Optimize and Evaluate

### MultinomialNB

### K- Fold evaluation

## SVC